## Description 
This is the webscrape code to grab the indicators from Ocean14 of any stocks listed in Brazilian´s capital marcket. \
The code will request the Indicators, it is a scoreboard that oceans14 list of all stock indicatores from 2002 to 2022 (current year of this document) \
First you will see a grup of functions to request the stock and after a function to list the stocks we want to see

### How did I get the below python code
1) login in Ocean14 \
2) go to the page you want to scrape \
4) access DevTools - Network tab \
5) search for the requests of the table "Quadro de Indicadores" -> example = ..../quadroIndicadores.aspx?empresa=vale \
6) right button and copy as cUrl bash \
7) past here -> https://curlconverter.com/python/ \
8) copy the generated python code

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# function to generate the CSV for each stock searched
def generateCSV(codigoAccao, dataFrame):
    dataFrame.to_csv("data_files/indicators_2005_2022_"+codigoAccao+".csv", index=False)

# function to prepare the json and generare a dataframe
def prepareDataSetinJson(thisJson):
    #print("quantidade", len(thisJson))
    for indicator in thisJson:
        papel= "Nome"
        if "empresa" in indicator:
            papel = indicator['empresa']
        elif "papel" in indicator:
            papel = indicator['papel']
        df = pd.DataFrame (indicator['saida'])
        #df['Name'] = papel
        df.insert(0,'Name',papel)
        generateCSV(papel, df)

# function to execute the request against Ocean14 and grab the indicators´s results for each stock in the list    
def getResposeBalancoDividendos(empresa, nome, numero, tipo):
    urlIndicadores ='https://www.oceans14.com.br/rendaVariavel/acoes/respostaAjax/quadroIndicadores.aspx'
    urlResultados  ='https://www.oceans14.com.br/rendaVariavel/acoes/respostaAjax/quadroResultados.aspx'
    urlIndicadBamco='https://www.oceans14.com.br/rendaVariavel/acoes/respostaAjax/quadroIndicadoresBanco.aspx'
    cookies = {
        '_ga': 'GA1.3.1208949115.1669038973',
        'uid': 'A8FC07BCAE48DFBCE3E5A0C83126C586',
        'origem': 'direto',
        '__gads': 'ID=83f34c6e8f234ad1:T=1669038967:S=ALNI_MaIjVfETrgPeCE-cKtEuSWr4lpojA',
        'Sessao': 'E5FEDFB7840E680466911B906336CB8B',
        'tipoUsuario': 'Usuario',
        'ASP.NET_SessionId': 'nn5zkmg1dockn0wt5qwvulzh',
        '_gid': 'GA1.3.978304171.1670090847',
        '_clck': '1ys97ww|1|f73|0',
        '__gpi': 'UID=000009d9d7318c1b:T=1669038967:RT=1670090846:S=ALNI_MZwYRYjY7kO0WG0uZnW2BHoMq6uoQ',
        'contadorViewsAcoes': '9',
    }
    headers = {
        'authority': 'www.oceans14.com.br',
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'accept-language': 'en-US,en;q=0.9',
        'referer': 'https://www.oceans14.com.br/acoes/{nome}/{empresa}{numero}/balanco-dividendos',
        'sec-ch-ua': '"Google Chrome";v="107", "Chromium";v="107", "Not=A?Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
        'x-requested-with': 'XMLHttpRequest',
    }

    if tipo == "indicadores":
        urlResponse = urlIndicadores
        params = {'empresa': empresa} 
    elif tipo == 'quadroIndicadoresBanco':
        urlResponse = urlIndicadBamco
        params = {'empresa': empresa}
    else:
        urlResponse = urlResultados
        params = {
            'empresa': empresa,
            'valores': 'Nominais',
            'apresentacao': 'Anual',
        }
    
    response = requests.get(
        urlResponse,
        params=params,
        cookies=cookies,
        headers=headers,
    )

    # Verify if request worked. 200 is success
    if response.status_code != 200:
        print("Error fetching page")
        exit()
    else:
        inJson = response.json()
        prepareDataSetinJson(inJson)



In [10]:
# list of stocks to get results
stocks = [
    ["vale","vale", "3", "indicadores"],
    ["petr", "petrobras", "", "indicadores"],
    ["klbn", "klabin", "", "indicadores"],
    ["bbas","banco-do-brasil", "3", "quadroIndicadoresBanco"]
]
for stock in stocks:
    empresa = acao[0]
    nome    = acao[1]
    numero  = acao[2]
    tipo    = acao[3]
    print("START EMPRESA-> % s NOME_PAPEL-> % s NUMERO_PAPEL-> % s QUADRO-> % s" % (empresa, nome, numero, tipo))
    getResposeBalancoDividendos(empresa, nome, numero, tipo)

print("END")
    

INICIA EMPRESA-> vale NOME_PAPEL-> vale NUMERO_PAPEL-> 3 QUADRO-> indicadores
quantidade 1
INICIA EMPRESA-> petr NOME_PAPEL-> petrobras NUMERO_PAPEL->  QUADRO-> indicadores
quantidade 2
INICIA EMPRESA-> klbn NOME_PAPEL-> klabin NUMERO_PAPEL->  QUADRO-> indicadores
quantidade 2
INICIA EMPRESA-> bbas NOME_PAPEL-> banco-do-brasil NUMERO_PAPEL-> 3 QUADRO-> quadroIndicadoresBanco
quantidade 1
FIM
